<a href="https://colab.research.google.com/github/sruthipsuresh/best-sellers-analysis/blob/main/BestSellers2021Scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Code heavily referenced https://www.datacamp.com/community/tutorials/amazon-web-scraping-using-beautifulsoup

In [35]:

!pip3 install beautifulsoup4

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import time
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests

no_pages = 1

def get_data(pageNo):
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

    r = requests.get('https://www.amazon.com/gp/bestsellers/2021/books/ref=zg_bsar_nav_b_0_1/'+str(pageNo)+'?ie=UTF8&pg='+str(pageNo), headers=headers)#, proxies=proxies)
    content = r.content
    soup = BeautifulSoup(content)
    #print(soup)

    alls = []
    for d in soup.findAll('div', attrs={'class':'a-section a-spacing-none aok-relative'}):
        #print(d)
        name = d.find('span', attrs={'class':'zg-text-center-align'})
        n = name.find_all('img', alt=True)
        #print(n[0]['alt'])
        author = d.find('a', attrs={'class':'a-size-small a-link-child'})
        rating = d.find('span', attrs={'class':'a-icon-alt'})
        users_rated = d.find('a', attrs={'class':'a-size-small a-link-normal'})
        price = d.find('span', attrs={'class':'p13n-sc-price'})
        link = d.find('a', attrs={'class':'a-link-normal'})['href']

        all1=[]

        if name is not None:
            #print(n[0]['alt'])
            all1.append(n[0]['alt'])
        else:
            all1.append("unknown-product")

        if author is not None:
            #print(author.text)
            all1.append(author.text)
        elif author is None:
            author = d.find('span', attrs={'class':'a-size-small a-color-base'})
            if author is not None:
                all1.append(author.text)
            else:
                all1.append('0')

        if rating is not None:
            #print(rating.text)
            all1.append(rating.text)
        else:
            all1.append('-1')

        if users_rated is not None:
            #print(price.text)
            all1.append(users_rated.text)
        else:
            all1.append('0')

        if price is not None:
            #print(price.text)
            all1.append(price.text)
        else:
            all1.append('0')
        
        if link is not None:
            all1.append(link)
        else:
            all1.append('0')

        alls.append(all1)
    return alls

results = []

for i in range(1, no_pages+1):
    results.append(get_data(i))
flatten = lambda l: [item for sublist in l for item in sublist]
df = pd.DataFrame(flatten(results),columns=['Book Name','Author','Rating','Customers_Rated', 'Price', 'Link'])
df.to_csv('amazon_products.csv', index=False, encoding='utf-8')

In [63]:
df.head(5) # Check to see if everything has a link.

,Book Name,Author,Rating,Customers_Rated,Price,Link
0,The Four Winds: A Novel,Kristin Hannah,4.5 out of 5 stars,"32,626",$17.39,/Four-Winds-Novel-Kristin-Hannah/dp/1250178606...
1,"The Boy, the Mole, the Fox and the Horse",Charlie Mackesy,4.9 out of 5 stars,"67,759",$14.03,/Boy-Mole-Fox-Horse/dp/0062976583/ref=zg_bsar_...
2,Atomic Habits: An Easy & Proven Way to Build G...,James Clear,4.8 out of 5 stars,"37,277",$11.98,/Atomic-Habits-Proven-Build-Break/dp/073521129...
3,Dog Man: Mothering Heights: From the Creator o...,Dav Pilkey,4.9 out of 5 stars,"2,413",$8.38,/Dog-Man-Mothering-Heights-Underpants/dp/13386...
4,I Love You to the Moon and Back,Amelia Hepworth,4.9 out of 5 stars,"29,555",$3.59,/I-Love-You-Moon-Back/dp/1589255518/ref=zg_bsa...


In [85]:
# Now, to find author bio, tags, summary

no_pages = 1
def get_data(link):
  headers = { 
      'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36', 
      'Accept' : 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8', 
      'Accept-Language' : 'en-US,en;q=0.5',
      'Accept-Encoding' : 'gzip', 
      'DNT' : '1', # Do Not Track Request Header 
      'Connection' : 'close'
      }

  r = requests.get('https://www.amazon.com'+link, headers=headers)#, proxies=proxies)
  content = r.content
  soup = BeautifulSoup(content, features="lxml")
  author_bio = soup.select("#productDescription_fullView")
  print(author_bio)
  

get_data('/Four-Winds-Novel-Kristin-Hannah/dp/1250178606/ref=zg_bsar_books_1?_encoding=UTF8&psc=1&refRID=R5F0E51S09YBSSQXVR83')


#results = []

#for i in range(1, no_pages+1):
#    results.append(get_data(i))
##flatten = lambda l: [item for sublist in l for item in sublist]
#df = pd.DataFrame(flatten(results),columns=['Author Bio'])
#df.to_csv('amazon_products.csv', index=False, encoding='utf-8')

[]
